In [ ]:
# Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os

In [ ]:
spark = SparkSession.builder.\
                    appName("spotify").\
                    config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").\
                    config("spark.mongodb.input.uri", "mongodb+srv://**********").\
                    config("spark.mongodb.output.uri", "mongodb+srv://**********").\
                    config("spark.network.timeout", "7200s").\
                    config("spark.executor.heartbeatInterval", "1200s").\
                    getOrCreate()

In [ ]:
spark

Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.147.238.15:7077 
 AppName 
 Databricks Shell

Configuration for accessing S3

In [ ]:
# Using Chandrish's access creds
ACCESS_KEY = "****"
SECRET_KEY = "****"
spark._jsc.hadoopConfiguration().set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1")
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", ACCESS_KEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET_KEY)

Reading Data from S3

#### Different Cluster settings
We tried to read data with two cluster settings:
- Cluster Setting 1: <br>
  9.1 LTS (includes Apache Spark 3.1.2, Scala 2.12) <br> 
  md5.large: 8gb memory 2 cores <br>
  Time: 7-8 mins (for reading data) <br>
  
- Cluster Setting 2: <br>
  10.3 (includes Apache Spark 3.2.1, Scala 2.12) <br>
  i3.2xlarge: 61 GB memory 8 cores <br>
  Time: 2.17 minutes

Reading playlists file, which is around 33GB and in json format

In [ ]:
# reading playlists file
playlist_file = 's3a://spotify-recommendation/playlists'
lyrics_file = 's3a://spotify-recommendation/lyrics'
play_df = spark.read.option("multiline", "true").json(playlist_file)

Reading the lyrics file, which has lyrics for 130k songs

In [ ]:
# reading lyrics file
lyrics_df = spark.read.json(lyrics_file)
lyrics_df = lyrics_df.select('track_uri', 'lyrics')

We have a nested json structure for the playlists file, so were exploding it to columns to make the join easier

In [ ]:
# reading elements of the playlist file
playlist_df = play_df.select('playlists', explode('playlists')).\
                    select(explode('col.tracks'),'col.name','col.pid',
                       'col.num_followers', 'col.num_tracks','col.duration_ms','col.num_artists').\
                        withColumnRenamed('col.num_followers', 'num_followers').\
                        withColumnRenamed('col.num_tracks', 'tracks').\
                        withColumnRenamed('col.duration_ms', 'duration_ms').\
                        withColumnRenamed('col.num_artists', 'num_artists').\
                        withColumnRenamed('col.pid', 'pid').\
                        withColumnRenamed('col.name', 'name').\
                        select('col.*','*').drop('col')

We are converting the lyrics string into list of tokenized words

In [ ]:
# tokenizing the lyrics
tokenizer = Tokenizer(inputCol="lyrics", outputCol="lyrics_words")
wordsData = tokenizer.transform(lyrics_df)

We are doing a left join of playlists data with lyrics data

In [ ]:
# joining the two datasets
master_data = playlist_df.join(wordsData, 'track_uri', 'left')

Data Quality Checks

In [ ]:
# checking the non null count for lyrics
master_data.where(master_data['lyrics'].isNotNull()).select('track_uri').distinct().count()

Out[10]: 137889

In [ ]:
# checking the null count for lyrics
master_data.where(master_data['lyrics'].isNull()).select('track_uri').distinct().count()

Out[128]: 2124403

In [ ]:
master_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+---+------------------+-----------+------+-------------+----------+-----------+-----------+------+------------+
 track_uri| album_name| album_uri| artist_name| artist_uri|duration_ms|pos| track_name| name| pid|num_followers|num_tracks|duration_ms|num_artists|lyrics|lyrics_words|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+---+------------------+-----------+------+-------------+----------+-----------+-----------+------+------------+
spotify:track:04M...| RKS|spotify:album:2aa...|Rainbow Kitten Su...|spotify:artist:4h...| 237493| 3| Cold Love| Sunday|381000| 1| 152| 31926486| 6| null| null|
spotify:track:0ES...|Birds In The Trap...|spotify:album:42W...| Travis Scott|spotify:artist:0Y...| 213863| 0| beibs in the trap|litty titty|115000| 1| 212| 51022342| 75| null| null|
spotify:track:0KK...| 24K Magic|spotify:album:4Pg...| Bruno Mars|spotify:artist:0d...| 206693| 0|That's What I Like| Car Vibes|477000| 1| 57| 13897129| 40| null| null|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+---+------------------+-----------+------+-------------+----------+-----------+-----------+------+------------+
only showing top 3 rows

### Setting up the connection string

In [ ]:
database = '*****'
collection = '*****'
user_name = '*****'
password = '*****'
address = '*****.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

Writing the files onto MongoDB: <br>
Took 37 minutes with Cluster Setting 2

In [ ]:
master_data.write.format("mongo").option("uri",connection_string).mode("append").save()
# it took 37 mins to write, when the notebook worked properly

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4394306349138779> in <module> 
 ----> 1 master_data . write . format ( "mongo" ) . option ( "uri" , connection_string ) . mode ( "append" ) . save ( ) 

 NameError : name 'master_data' is not defined

Reading the data back from MongoDB

In [ ]:
# reading the data back to check
df = spark.read.format('mongo').option("uri",connection_string).load()

Data Upload Checks

In [ ]:
# checking the non null count and compare with previous result above
df.where(df['lyrics'].isNotNull()).select('track_uri').distinct().count()

Out[8]: 137889

In [ ]:
# seeing how lyrics looks in the database
df.where(df['lyrics'].isNotNull()).show(1000)

+--------------------+--------------------+--------------------+-----------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------+-------------+----------+------+---+--------------------+--------------------+
 _id| album_name| album_uri| artist_name| artist_uri|duration_ms| lyrics| lyrics_words| name|num_artists|num_followers|num_tracks| pid|pos| track_name| track_uri|
+--------------------+--------------------+--------------------+-----------------+--------------------+-----------+--------------------+--------------------+--------------------+-----------+-------------+----------+------+---+--------------------+--------------------+
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 46045590|All we shared was...|[all, we, shared,...| Sam| 54| 7| 181| 62734| 27| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 41307347|All we shared was...|[all, we, shared,...| Chill jams| 124| 2| 190|514425| 20| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 55100801|All we shared was...|[all, we, shared,...| Playlist| 103| 1| 242|718221| 46| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 18944057|All we shared was...|[all, we, shared,...| Yeah| 31| 1| 81|452925| 54| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 33700896|All we shared was...|[all, we, shared,...| Seventeen| 16| 1| 158|628746| 37| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 53670133|All we shared was...|[all, we, shared,...| earworms ~~| 56| 1| 219|378192|131| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 55808261|All we shared was...|[all, we, shared,...| Throwback Jams| 30| 1| 247|522391|243| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 31984959|All we shared was...|[all, we, shared,...| Viajar| 21| 1| 123|805358| 49| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 23208653|All we shared was...|[all, we, shared,...| Rock| 9| 2| 99| 16994| 10| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 30209731|All we shared was...|[all, we, shared,...| Bit of Everything| 68| 1| 131|429248| 41| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 17145562|All we shared was...|[all, we, shared,...| Punk| 40| 2| 96|434333| 38| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 48059426|All we shared was...|[all, we, shared,...| art | 131| 1| 210|515425| 42| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 48303250|All we shared was...|[all, we, shared,...| Perfect Playlist| 201| 2| 207|541589|142| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 44283379|All we shared was...|[all, we, shared,...| Gaming| 65| 2| 176|440012| 16| Perfect World|spotify:track:00W...|
{621bb23d5f573454...| Billy Talent II|spotify:album:0cT...| Billy Talent|spotify:artist:08...| 10102981|All we shared was...|[all, we, shared,...| bored| 37| 2| 46|440800| 24| Perfect World|spotify:track:00W...|
{621bb23d5f573454...|Sing Sing Death H...|spotify:album:4Vr